In [ ]:
import datetime
from pathlib import Path
import sys

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.utils.frame_widget import FrameWidget
from skrgbd.data.io import imgio
from skrgbd.utils.logging import logger, tqdm
from skrgbd.devices.realsense import RealSense

In [ ]:
today = datetime.datetime.now().strftime('%y_%m_%d')
logfile = f'/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/calibration/logs/{today}.log'
logger.prepare_for_scanning(logfile)

In [ ]:
cam = RealSense(rgb_enabled=False)
cam.start()

In [ ]:
cam.set_ir_exposure(6000)
cam.set_ir_gain(16)
cam.laser_on()
cam.set_laser_power(360)

In [ ]:
frame = cam.snap_frame(compressed=True)
widget = FrameWidget(lambda: frame['depth'], (8, 5), cmap='jet', vmin=500, vmax=1200)
# widget = FrameWidget(lambda: frame['ir'], (8, 5), cmap='gray', vmin=0, vmax=1)
widget.update()
widget.image

In [ ]:
cam_name = 'real_sense'
img_dir = f'/mnt/data/sk3d/calibration/images/depth_wall_{cam_name}/{cam_name}_raw'
for mode in ['ir', 'depth']:
    Path(f'{img_dir}/{mode}').mkdir(parents=True, exist_ok=True)

img_i = 0
while True:
    frame = cam.snap_frame(compressed=True)
    widget.update()
    imgio.write_png(f'{img_dir}/ir/{img_i:06}.png', frame['ir'], optimize=False, compress_level=0)
    imgio.write_png(f'{img_dir}/depth/{img_i:06}.png', frame['depth'], optimize=False, compress_level=0)
    img_i += 1